In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
from fundl.layers.normalizing_flow import K_planar_flows
from fundl.weights import add_K_planar_flow_params
import autograd.numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact, FloatSlider, IntSlider

# Normalizing flows

This is a demo notebook that shows how normalizing flows work. 

## What are normalizing flows?

They are nothing more than transformations from one probability distribution to another probability distribution.

Those of us who have studied probability before have likely seen an example. For example, if 

$$X \sim Normal$$

and if 

$$Y = e^X$$

Then

$$Y \sim LogNormal$$

Here, the exponentiation is an **invertible transform** on a probability distribution.

Normalizing Flows are all about using invertible transforms to take a simple probability distribution and convert it into a different, complicated probability distribution.

To learn more about normalizing flows, check out the following pages:

- http://akosiorek.github.io/ml/2018/04/03/norm_flows.html
- https://docs.pymc.io/notebooks/normalizing_flows_overview.html

## Planar Flows

Planar flows are a type of normalizing flow. According to the [2016 paper on it](https://arxiv.org/abs/1505.05770), planar flows can be thought of as "squeezers and dilators" of probability distribution space. Some parts of the probability distribution are squeezed together, while other parts of the probability distribution are dilated. 

Planar flows are described by the following set of transformations:

$$f(\textbf{z}) = \textbf{z} + \textbf{u}\textit{h}(\textbf{w}^T\textbf{z}+b)$$

The free parameters in the model are $\textbf{w}$, $\textbf{u}$ and $b$, which we can control. $\textbf{w}$ and $\textbf{u}$ are bolded because they are vectors, while $b$ is a scalar, and hence is not bolded. $h$ is a nonlinear transformation; in this notebook, I have stuck to the $\text{tanh}$ function.

In the following cell, we will visualize what planar flows do to probability distributions.

First off, some utility functions to create the parameters for planar flows.

In [7]:
from autograd.numpy.random import normal
def add_planar_flow_params(params, name, dim, p):
    """
    Instantiates parameters for a planar flow transformation.
    
    Modified from fundl. 
    """
    params[name] = dict()
    params[name]["w"] = normal(size=(dim, 1), **p)
    params[name]["b"] = normal(**p)
    params[name]["u"] = normal(size=(dim, 1), **p)
    return params


def add_K_planar_flow_params(params, K, dim, p):
    """
    Updates a params dictionary with K planar flow parameters.
    
    Modified from fundl.
    """
    Ks = []
    for i in range(K):
        name = f"planar_flow_{i}"
        params = add_planar_flow_params(params, name, dim, p)
        Ks.append(name)
    return params, Ks

### Instructions: Univariate Distributions

In the cell below, we start with 1000 draws from a standard normal distribution. This is the `original`. We then pass it through `K` planar flows.

Run the cell below, and then play around with:

1. Changing the `loc`ation, holding all else equal.
2. Changing `K`, holding all else equal.

The `scale` parameter controls the amount of noise in the initialization. Having zero noise lets us explore the effect of `loc` and `K` on the transformed probability distribution.

In [9]:
a = np.random.normal(size=(1000, 1))

@interact(
    loc=FloatSlider(min=-10, max=10), 
    scale=FloatSlider(min=0.001, max=20),
    K=IntSlider(min=1, max=10)
)
def plot_original_transformed(loc, scale, K):
    p = dict(loc=loc, scale=scale)
    params = dict()
    params, K_names = add_K_planar_flow_params(params, K=K, dim=1, p=p)
    z_tfm, ldj = K_planar_flows(params, a, K_names)
    sns.kdeplot(*z_tfm.T, label='transformed')
    sns.kdeplot(*a.T, label='original')

interactive(children=(FloatSlider(value=0.0, description='loc', max=10.0, min=-10.0), FloatSlider(value=0.001,…

We initialized with location parameters at zero. With this, the planar transformed distributions are identical to the original. As we change the location, though, the probability distribution starts to shift. 

If you added noise (changing the `scale` parameter), you can then get a feel for the rich family of probability distributions that are available.

### Instructions: Bivariate Distributions

Same instructions as for the univariate distributions: Change `loc` and/or `scale`, either together or individually, and visualize what happens to the isotropic bivariate Gaussian.

In [10]:
a2 = np.random.multivariate_normal(mean=[0,0], cov=[[1, 0],[0, 1]], size=1000)


@interact(
    loc=FloatSlider(min=-10, max=10), 
    scale=FloatSlider(min=0.001, max=20),
    K=IntSlider(min=1, max=10)
)
def plot_multivariate_nf(loc, scale, K):
    p = dict(loc=loc, scale=scale)

    params = dict()
    params, K_names = add_K_planar_flow_params(params, K=K, dim=2, p=p)

    z_tfm, ldj = K_planar_flows(params, a2, K_names)
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4), sharex=True, sharey=True)
    axes[1].scatter(*z_tfm.T, alpha=0.1)
    axes[1].set_title('transformed')

    axes[0].scatter(*a2.T, alpha=0.1)
    axes[0].set_title('original')

interactive(children=(FloatSlider(value=0.0, description='loc', max=10.0, min=-10.0), FloatSlider(value=0.001,…